### Import Modules

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.stats import shapiro, normaltest
from tpot import TPOTClassifier
from hpsklearn import HyperoptEstimator, pca, min_max_scaler, standard_scaler
from hpsklearn import xgboost_classification, random_forest, ada_boost, gradient_boosting, extra_trees
from hpsklearn import svc, svc_linear, svc_rbf, svc_poly, svc_sigmoid, liblinear_svc
from hpsklearn import any_classifier
from hpsklearn import any_preprocessing
from hyperopt import tpe, hp
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
import ngboost as ngb
import optuna
from bayes_opt import BayesianOptimization
from skopt import BayesSearchCV
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score, accuracy_score, f1_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report,confusion_matrix,plot_confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.feature_selection import RFE, SelectFromModel, RFECV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif
from matplotlib import pyplot

import warnings
warnings.filterwarnings('ignore')

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


### Load and Clean Data

In [2]:
df_train = pd.read_csv('Data/cancer_classification.csv')
#df_train = pd.read_csv('Data/higgs_boson_training.csv')

In [3]:
df_train.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,benign_0__mal_1
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [5]:
df_train.shape

(569, 31)

In [6]:
df_train.isnull().sum().sum()

0

In [7]:
# calculate duplicates
dups = df_train.duplicated()
#print(dups)
# report if there are any duplicates
print(dups.any())

False


In [8]:
df_train["benign_0__mal_1"].value_counts()
#df_train["Label"].value_counts()

1    357
0    212
Name: benign_0__mal_1, dtype: int64

In [10]:
class_names = {0:'Benign', 1:'Malignant'}
print(df_train.benign_0__mal_1.value_counts().rename(index = class_names))
#df_train['Label'] = df_train['Label'].map({'b':0,'s':1})
#df_train["Label"].value_counts()

Malignant    357
Benign       212
Name: benign_0__mal_1, dtype: int64


In [11]:
Malignant = df_train[df_train['benign_0__mal_1']==1]
Benign = df_train[df_train['benign_0__mal_1']==0]
outlier_fraction = len(Malignant)/float(len(Benign))
print (outlier_fraction)

1.6839622641509433


In [12]:
X = df_train.drop(['benign_0__mal_1'],axis=1)
y = df_train['benign_0__mal_1']

In [13]:
#y = LabelEncoder().fit_transform(y)

### Train Test Split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)

In [15]:
X_train.shape

(398, 30)

In [16]:
X_test.shape

(171, 30)

### Find Best Algorithm

#### Find Best Algorithm with Best Params Using HyperoptEstimator AutoML

In [20]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
accuracy = 0
best_model = None
#for i in range(100):
    #print (i)
model = HyperoptEstimator(  classifier= any_classifier('cla'), 
                                preprocessing= any_preprocessing('pre'), 
                                algo=tpe.suggest, 
                                max_evals=50, 
                                trial_timeout=5000)
    # perform the search
model.fit(X_train, y_train)
acc = model.score(X_test, y_test)
    #if acc > accuracy:
        #accuracy = acc
        #best_model = model
        #print (accuracy)

fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

Job Started at: 2020-11-28 17:19:40.251060
100%|██████████| 50/50 [00:04<00:00,  4.53s/trial, best loss: 0.0]
Execution time :  0:03:05.734596
Job Ended at: 2020-11-28 17:22:45.985656


In [21]:
#summarize the best model
best_model = model.best_model()
print("Accuracy: %.4f" % best_model.score(X_test, y_test))
print(best_model.best_model())

Accuracy: 0.953
{'learner': ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=4,
                     max_features=0.7975833610618567, max_leaf_nodes=None,
                     max_samples=None, min_impurity_decrease=0.0,
                     min_impurity_split=None, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=626, n_jobs=1, oob_score=False,
                     random_state=4, verbose=False, warm_start=False), 'preprocs': (StandardScaler(copy=True, with_mean=True, with_std=True),), 'ex_preprocs': ()}


In [ ]:
init_time = datetime.now()
model = XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.50029110018247, colsample_bynode=1,
              colsample_bytree=0.9371080486138555, gamma=0.00014999673883672553,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.15017743323882513, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=4,
              reg_alpha=6.402887715271464e-05, reg_lambda=1.0516334360090362,
              scale_pos_weight=1, seed=4, subsample=0.9995865341563067,
              tree_method='exact', validate_parameters=1, verbosity=None)

model.fit(X_train, y_train,verbose=False,
            early_stopping_rounds=10
            #eval_set=[(X_valid,y_valid)]
         )

prediction = model.predict(X_test)
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))

In [ ]:
print(classification_report(y_test, prediction))

#### Find Best Algorithm with Best Params Using TPOT AutoML

In [ ]:
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [22]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")

# define model evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search
#model = TPOTClassifier(generations=5, population_size=50, cv=cv, scoring='accuracy', 
                       #verbosity=2, random_state=1, n_jobs=-1)
tpot = TPOTClassifier(max_time_mins=120, cv=cv, scoring='accuracy', 
                       verbosity=2, random_state=1, n_jobs=-1)
# perform the search
tpot.fit(X_train, y_train)
# export the best model
#tpot.export('tpot_Otto_best_model.py')
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

Job Started at: 2020-11-28 17:24:17.836220



Generation 1 - Current best internal CV score: 0.9756837606837607

Generation 2 - Current best internal CV score: 0.9756837606837607

Generation 3 - Current best internal CV score: 0.9773290598290598

Generation 4 - Current best internal CV score: 0.9781837606837607

Generation 5 - Current best internal CV score: 0.9832051282051283

Generation 6 - Current best internal CV score: 0.9832051282051283

Generation 7 - Current best internal CV score: 0.9832051282051283

Generation 8 - Current best internal CV score: 0.9840384615384615

Generation 9 - Current best internal CV score: 0.9840384615384615

Generation 10 - Current best internal CV score: 0.9840384615384615

Generation 11 - Current best internal CV score: 0.9840384615384615

Generation 12 - Current best internal CV score: 0.9840384615384615

Generation 13 - Current best internal CV score: 0.9840384615384615

121.65 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurel

In [23]:
tpot.fitted_pipeline_

Pipeline(memory=None,
         steps=[('zerocount', ZeroCount()),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('stackingestimator-1',
                 StackingEstimator(estimator=XGBClassifier(base_score=0.5,
                                                           booster='gbtree',
                                                           colsample_bylevel=1,
                                                           colsample_bynode=1,
                                                           colsample_bytree=1,
                                                           gamma=0, gpu_id=-1,
                                                           importance_type='gain',
                                                           interaction_constraints=''...
                               batch_size='auto', beta_1=0.9, beta_2=0.999,
                               early_stopping=False, epsilon=1e-08

In [24]:
tpot.score(X_test, y_test)

0.9707602339181286

#### Find Best Algorithm with Best Params Using PyCaret

In [17]:
from pycaret.classification import * 
from pycaret.datasets import get_data

##### PyCaret Setup

In [18]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
#fold_strategy = 'kfold' Default = 'stratifiedkfold'
clf = setup(data = df_train, target = 'benign_0__mal_1', session_id=123,
            data_split_stratify = True,
            #normalize = True, 
            #normalize_method = 'minmax',
            #transformation = True, 
            #transformation_method = 'quantile',
            #pca = True,
            #pca_method = 'incremental',
            #feature_selection = True,
            #feature_selection_threshold = 0.5,
            #feature_selection_method = 'boruta',
            #remove_outliers = True,
            #outliers_threshold = 0.05,
            #fix_imbalance = True
           )
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

,Description,Value
0,session_id,123
1,Target,benign_0__mal_1
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(569, 31)"
5,Missing Values,False
6,Numeric Features,30
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


Execution time :  0:00:14.740240
Job Ended at: 2020-11-28 17:17:19.416215


##### Finding optimal model

In [19]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
#top3_models = compare_models(n_select = 3)
best = compare_models()
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9675,0.9885,0.9760,0.9730,0.9743,0.9302,0.9310,0.2510
lightgbm,Light Gradient Boosting Machine,0.9675,0.9885,0.9840,0.9656,0.9743,0.9301,0.9321,0.0770
rf,Random Forest Classifier,0.9674,0.9886,0.9880,0.9618,0.9745,0.9292,0.9307,0.2770
catboost,CatBoost Classifier,0.9649,0.9899,0.9840,0.9619,0.9725,0.9241,0.9256,4.8040
gbc,Gradient Boosting Classifier,0.9574,0.9826,0.9760,0.9576,0.9663,0.9082,0.9098,0.1520
lda,Linear Discriminant Analysis,0.9574,0.9872,1.0000,0.9387,0.9678,0.9056,0.9117,0.0210
xgboost,Extreme Gradient Boosting,0.9524,0.9864,0.9680,0.9577,0.9623,0.8975,0.8993,0.0960
qda,Quadratic Discriminant Analysis,0.9522,0.9885,0.9560,0.9689,0.9619,0.8978,0.8994,0.0280
ridge,Ridge Classifier,0.9499,0.0000,0.9960,0.9309,0.9620,0.8890,0.8945,0.0250
ada,Ada Boost Classifier,0.9499,0.9881,0.9800,0.9444,0.9614,0.8902,0.8935,0.0850


Execution time :  0:01:16.600605
Job Ended at: 2020-11-28 17:18:46.497164


In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
best_model = finalize_model(best)
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

In [ ]:
print (best_model)

##### Tune Model

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
#tuned_model = tune_model(best_model, optimize = 'AUC') # 'Recall'
tuned_model = tune_model(best_model, optimize = 'F1')
print(tuned_model)
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))
print (f"Job Ended at: {fin_time}")

##### Predictions

In [ ]:
predictions = predict_model(tuned_model)

### Find Best Params of Gradient Boosting 

#### XGBoost Hyperparameter Optimization

##### skopt Optimization

In [ ]:
#xgb_params["n_estimators"]=(10, 100),
#xgb_params['min_child_weight']=(1, 20),  
#xgb_params['scale_pos_weight']= (1.0, 16.0),  For Imbalanced

# define search space
xgb_params = dict()
#params['booster'] = ['gbtree', 'dart']
#params["grow_policy"]= ["depthwise", "lossguide"]
xgb_params['learning_rate'] = (1e-7, 1.0, 'log-uniform')
xgb_params['min_child_weight'] = (1e-16, 1e5, 'log-uniform')
xgb_params['subsample'] = (0.5, 1.0)
xgb_params['max_depth'] = (2, 20)
xgb_params['colsample_bylevel'] = (0.5, 1.0)
xgb_params['colsample_bytree'] = (0.5, 1.0)
xgb_params['colsample_bynode'] = (0.5, 1.0)
#xgb_params['max_delta_step'] = (0.1, 10)
#xgb_params['bagging_temperature'] = (0.5, 1.0)
#xgb_params['random_strength'] = (0.0, 100)
xgb_params['reg_alpha'] = (0.0, 100)
xgb_params['reg_lambda'] = (0.0, 100)
#xgb_params['gamma'] = (0.0, 100)

dart_param = dict()
dart_param["sample_type"]= ["uniform", "weighted"],
dart_param["normalize_type"]= ["tree", "forest"],
dart_param["rate_drop"]= (1e-8, 1.0),
dart_param["skip_drop"]= (1e-8, 1.0)
           
#if xgb_params["booster"] == "dart":
     #xgb_params.update(dart_param)

In [ ]:
#For Multi-Classification - multi:softmax
#For Binary Classificayion - binary:logistic
xgb_model = XGBClassifier(
        objective='multi:softmax',
        booster='dart'
        n_estimators=10000, # use large n_estimators deliberately to make use of the early stopping
        n_jobs=-1
    )
fit_params = {
    'early_stopping_rounds':30,  
    'eval_set':[(X_test, y_test)],
    'verbose':0
}

In [ ]:
# define evaluation
#xgb_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=101)
# define the search
xgb_search = BayesSearchCV(estimator=xgb_model, search_spaces=xgb_params, fit_params=fit_params,
                           cv=5, n_iter=50,
                           verbose=0, n_jobs=-1,
                           )

In [ ]:
# callback handler
def on_step(optim_result):
    score = xgb_search.best_score_
    print (f"{datetime.now()}: Best Score- {score:.4f}")
    if score >= 0.98:
        print('Interrupting!')
        return True

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
# perform the search
xgb_search.fit(X_train, y_train, callback=on_step)
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))

##### Bayesian Optimization

In [ ]:
xgb_pbounds = {
    'learning_rate': (1e-7, 1.0),
    'min_child_weight': (1e-16, 1e5),
    'max_depth': (2,20),
    'subsample': (0.3, 1.0),
    'colsample_bylevel': (0.3, 1.0), 
    'colsample_bytree': (0.3, 1.0),
    'colsample_bynode': (0.3, 1.0),
    'reg_alpha': (0, 100),
    'reg_lambda': (0, 100)
} 

In [ ]:
def xgboost_hyper_param(learning_rate,
                        min_child_weight,
                        max_depth,
                        subsample,
                        colsample_bylevel, 
                        colsample_bytree,
                        colsample_bynode,
                        reg_alpha,
                        reg_lambda):

    max_depth = int(max_depth)

    clf = XGBClassifier(
        objective= "multi:softmax",
        booster = "dart",
        n_estimators = 10000,
        learning_rate=learning_rate,
        min_child_weight= min_child_weight,
        max_depth=max_depth,
        subsample= subsample,  
        colsample_bylevel= colsample_bylevel, 
        colsample_bytree= colsample_bytree,
        colsample_bynode= colsample_bynode,
        reg_alpha= reg_alpha,
        reg_lambda= reg_lambda
        )
    
    fit_params = {
        'early_stopping_rounds':30,
        'eval_set':[(X_test, y_test)],
        'verbose':0
    }

    #cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=101)
    cv_result = np.mean(cross_val_score(estimator=clf, X=X_train, y=y_train, fit_params=fit_params, 
                                        cv=5, scoring='accuracy', verbose = 0))
    return cv_result

xgb_optimizer = BayesianOptimization(
    f=xgboost_hyper_param,
    pbounds=xgb_pbounds,
    #verbose=2,
    random_state=1,
)

In [ ]:
init_time = datetime.now()
print (f"Job Started at: {init_time}")
init_points = 8 
n_iter = 50 #The more steps the more likely to find a good maximum you are.
xgb_optimizer.maximize(n_iter=n_iter, init_points=init_points)
fin_time = datetime.now()
print("Execution time : ", (fin_time-init_time))